In [51]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, SimpleRNN, GRU, LSTM, Bidirectional, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [3]:
df = pd.read_csv("tcc_ceds_music.csv")
df

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,0.000598,0.000598,0.048857,0.017104,0.263751,0.000598,0.039288,0.000598,0.000598,0.000598,0.000598,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,0.001284,0.001284,0.027007,0.001284,0.001284,0.001284,0.118034,0.001284,0.212681,0.051124,0.001284,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,0.002770,0.002770,0.002770,0.158564,0.250668,0.002770,0.323794,0.002770,0.002770,0.002770,0.002770,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,0.001548,0.021500,0.001548,0.411536,0.001548,0.001548,0.001548,0.129250,0.001548,0.001548,0.081132,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,0.001350,0.001350,0.001350,0.463430,0.001350,0.001350,0.001350,0.001350,0.001350,0.029755,0.001350,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,0.001350,0.001350,0.001350,0.001350,0.001350,0.391651,0.001350,0.435089,0.001350,0.001350,0.001350,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,0.001284,0.001284,0.001284,0.066324,0.203889,0.318910,0.058152,0.134955,0.001284,0.001284,0.040811,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,0.001504,0.039755,0.001504,0.035401,0.001504,0.356685,0.001504,0.068684,0.001504,0.001504,0.001504,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,0.001196,0.048359,0.001196,0.001196,0.001196,0.492434,0.103614,0.001196,0.202659,0.001196,0.070867,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [4]:
#remove unneccessary data from dataset
df = df[['lyrics', 'genre']]
df

,lyrics,genre
0,hold time feel break feel untrue convince spea...,pop
1,believe drop rain fall grow believe darkest ni...,pop
2,sweetheart send letter goodbye secret feel bet...,pop
3,kiss lips want stroll charm mambo chacha merin...,pop
4,till darling till matter know till dream live ...,pop
...,...,...
28367,cause fuck leave scar tick tock clock come kno...,hip hop
28368,minks things chain ring braclets yap fame come...,hip hop
28369,get ban get ban stick crack relax plan attack ...,hip hop
28370,check check yeah yeah hear thing call switch g...,hip hop


In [5]:
#train test split
# Splitting the data into features (X) and target (y)
X = df.drop(columns=['genre'])
y = df['genre']

# Performing train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Assuming y_train and y_test are your target variables
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.transform(y_test)

In [7]:
#reference: https://www.tensorflow.org/text/guide/word_embeddings
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
vectorize_layer.adapt(X_train)

In [8]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(7)
])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [33]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    callbacks = [callback])

Epoch 1/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.5751 - loss: 0.2791 - val_accuracy: 0.3514 - val_loss: 0.3846
Epoch 2/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 58s 82ms/step - accuracy: 0.6180 - loss: 0.2591 - val_accuracy: 0.3507 - val_loss: 0.3968


Trying RNN:

In [30]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GRU(32, return_sequences=True),
  SimpleRNN(16),
  Dense(7)
])

In [31]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [32]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    callbacks= [callback])

Epoch 1/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.2327 - loss: 0.4333 - val_accuracy: 0.2463 - val_loss: 0.3890
Epoch 2/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 50s 70ms/step - accuracy: 0.2641 - loss: 0.3816 - val_accuracy: 0.3082 - val_loss: 0.3716
Epoch 3/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 39s 55ms/step - accuracy: 0.3326 - loss: 0.3613 - val_accuracy: 0.3279 - val_loss: 0.3670
Epoch 4/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.3700 - loss: 0.3481 - val_accuracy: 0.3417 - val_loss: 0.3619
Epoch 5/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 52s 74ms/step - accuracy: 0.4461 - loss: 0.3246 - val_accuracy: 0.3588 - val_loss: 0.3614
Epoch 6/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.5223 - loss: 0.2985 - val_accuracy: 0.3642 - val_loss: 0.3659


LSTM RNN:

In [35]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  LSTM(32),
  Dense(7)
])

In [36]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [37]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    callbacks= [callback])

Epoch 1/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2253 - loss: 0.4224 - val_accuracy: 0.2463 - val_loss: 0.3875
Epoch 2/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2470 - loss: 0.3856 - val_accuracy: 0.2388 - val_loss: 0.3981


Bidirectional:

In [41]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  Bidirectional(LSTM(32, return_sequences=True), input_shape=(5,7)),
  Bidirectional(LSTM(16)),
  Dense(7)
])

d:\Projects\School Projects\Spring 2024\CS-4375\final project\env\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [42]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    callbacks= [callback])

Epoch 1/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.2289 - loss: 0.4215 - val_accuracy: 0.2516 - val_loss: 0.3876
Epoch 2/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 40s 56ms/step - accuracy: 0.2567 - loss: 0.3829 - val_accuracy: 0.2967 - val_loss: 0.3737
Epoch 3/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 57s 81ms/step - accuracy: 0.3163 - loss: 0.3650 - val_accuracy: 0.3230 - val_loss: 0.3686
Epoch 4/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 54s 76ms/step - accuracy: 0.4093 - loss: 0.3389 - val_accuracy: 0.3619 - val_loss: 0.3554
Epoch 5/15
710/710 ━━━━━━━━━━━━━━━━━━━━ 67s 94ms/step - accuracy: 0.4797 - loss: 0.3158 - val_accuracy: 0.3602 - val_loss: 0.3589


Test with different params:

In [52]:
embedding_dim=100

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  Bidirectional(LSTM(128, return_sequences=True)),
  Dropout(.5),
  Bidirectional(LSTM(64)),
  BatchNormalization(),
  Dense(64, activation="relu"),
  Dropout(.5),
  Dense(7, activation="softmax")
])

In [53]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [54]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Stop after 3 epochs if no improvement
    restore_best_weights=True
)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    callbacks= [callback])

Epoch 1/50
710/710 ━━━━━━━━━━━━━━━━━━━━ 119s 161ms/step - accuracy: 0.2321 - loss: 1.8370 - val_accuracy: 0.2793 - val_loss: 1.8575
Epoch 2/50
710/710 ━━━━━━━━━━━━━━━━━━━━ 111s 156ms/step - accuracy: 0.3768 - loss: 1.5868 - val_accuracy: 0.3494 - val_loss: 1.6627
Epoch 3/50
710/710 ━━━━━━━━━━━━━━━━━━━━ 142s 200ms/step - accuracy: 0.4816 - loss: 1.3749 - val_accuracy: 0.3385 - val_loss: 1.6725
Epoch 4/50
710/710 ━━━━━━━━━━━━━━━━━━━━ 115s 162ms/step - accuracy: 0.5614 - loss: 1.1818 - val_accuracy: 0.3329 - val_loss: 1.8854
Epoch 5/50
710/710 ━━━━━━━━━━━━━━━━━━━━ 128s 181ms/step - accuracy: 0.6351 - loss: 1.0149 - val_accuracy: 0.3357 - val_loss: 1.9689


Do hyperparameter optimization

In [56]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = keras.Sequential([
        vectorize_layer,
        Embedding(vocab_size, hp.Int('embedding_dim', min_value=32, max_value=256, step=32), name="embedding"),
        Bidirectional(LSTM(hp.Int('units_1', min_value=32, max_value=256, step=32), return_sequences=True)),
        Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)),
        Bidirectional(LSTM(hp.Int('units_2', min_value=32, max_value=256, step=32))),
        BatchNormalization(),
        Dense(hp.Int('units_3', min_value=32, max_value=256, step=32), activation="relu"),
        Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(7, activation="softmax")
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

C:\Users\Nevin\AppData\Local\Temp\ipykernel_21156\2861290751.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [58]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Stop after 3 epochs if no improvement
    restore_best_weights=True
)

# Define tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='text_classification')

# Start the search for the best hyperparameter configuration
tuner.search(X_train, y_train,
             validation_data=(X_test, y_test),
             epochs=50,
             callbacks=[callback])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 40m 41s]
val_accuracy: 0.34140968322753906

Best val_accuracy So Far: 0.35885462164878845
Total elapsed time: 06h 42m 23s


In [62]:
print(best_hps.values)

{'embedding_dim': 224, 'units_1': 32, 'dropout_1': 0.2, 'units_2': 64, 'units_3': 224, 'dropout_2': 0.2}
